In this notebook we explore the `compare` function and the `Report` class offered by `ranx`.

First of all we need to install [ranx](https://github.com/AmenRa/ranx)

Mind that the first time you run any ranx' functions they may take a while as they must be compiled first

In [ ]:
!pip install -U ranx

Download the data we need

In [ ]:
import os
import requests

for file in ["qrels", "run_1", "run_2", "run_3", "run_4", "run_5"]:
    os.makedirs("notebooks/data", exist_ok=True)

    with open(f"notebooks/data/{file}.trec", "w") as f:
        master = f"https://raw.githubusercontent.com/AmenRa/ranx/master/notebooks/data/{file}.trec"
        f.write(requests.get(master).text)

Compare

In [ ]:
from ranx import Qrels, Run, compare

In [ ]:
# Let's load qrels and runs from files
qrels = Qrels.from_file("notebooks/data/qrels.trec")

run_1 = Run.from_file("notebooks/data/run_1.trec")
run_2 = Run.from_file("notebooks/data/run_2.trec")
run_3 = Run.from_file("notebooks/data/run_3.trec")
run_4 = Run.from_file("notebooks/data/run_4.trec")
run_5 = Run.from_file("notebooks/data/run_5.trec")

In [ ]:
# Compares different runs and performs statistical tests (Fisher's Randomization test)
report = compare(
    qrels,
    runs=[run_1, run_2, run_3, run_4, run_5],
    metrics=["map@100", "mrr@100", "ndcg@10"],
    max_p=0.01,  # P-value threshold
    # Use `student` for Two-sided Paired Student's t-Test (fast / default),
    # `fisher` for Fisher's Randomization Test (slow),
    # or `tukey` for Tukey's HSD test (fast)
    stat_test="student"
)

# The comparison results are saved in a Report instance,
# which provides handy functionalities such as tabular formatting
# (superscripts denote statistical significance differences)
report

In [ ]:
# You can change the number of shown digits as follows
report.rounding_digits = 4
report

In [ ]:
# You can show percentages insted of digits
# Note that the number of shown digits is based on
# the `rounding_digits` attribute, try changing it
report.rounding_digits = 3
report.show_percentages = True
report

In [ ]:
# `rounding_digits` and `show_percentages` can be passed directly when
# calling `compare`
report = compare(
    qrels,
    runs=[run_1, run_2, run_3, run_4, run_5],
    metrics=["map@100", "mrr@100", "ndcg@10"],
    max_p=0.01,  # P-value threshold
    rounding_digits=3,
    show_percentages=True,
)

report

In [ ]:
# A Report can also be exported as LaTeX table ready for scientific publications
# Again you can control the number of digits and showing percentages
report.rounding_digits = 4
report.show_percentages = False

print(report.to_latex())

In [ ]:
report.rounding_digits = 3
report.show_percentages = True

print(report.to_latex())

In [ ]:
# A Report also keep track of the number of win, tie, and loss
report.win_tie_loss[("model_1", "model_2")]

In [ ]:
# You can extract a Report data and convert it to a Python Dictionary
# or save it as JSON file
import json  # Just for pretty printing

print(json.dumps(report.to_dict(), indent=4))
report.save("report.json")